In [3]:
import pickle
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
import numpy as np
import pandas as pd
from rdkit.Chem.rdmolops import RDKFingerprint




with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb.pickle', 'rb') as f:
    sms = pickle.load(f)

In [22]:
def generate_FPs(smiles, radius=2, length=300):
    '''
    smiles df should contain smiles column and cid column
    returns morganFP
    todo: add also searching by dname
    '''
    from rdkit.Chem import MolFromSmiles
    from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
    
    
    holder = np.zeros((len(smiles), length), dtype=np.uint8)
    for e,i in enumerate(smiles.iteritems()):
        a = MolFromSmiles(i[1])
        #holder[e] = np.asarray(GetMorganFingerprintAsBitVect(a, radius=radius, nBits=length), dtype=np.uint8)
        holder[e] = np.asarray(RDKFingerprint(a, fpSize=length), dtype = np.uint8 )
        
    return holder

In [23]:
import pandas as pd
smiles = pd.DataFrame(sms)

fps_topo = generate_FPs(sms)
df = pd.DataFrame(fps_topo, index = sms.index)
fin = pd.concat((smiles, df), axis=1)
fin.shape

(4153, 301)

In [11]:
#fin.to_csv('/tf/notebooks/code_for_pub/smiles_file/fps_morga.csv', header=True, index=True)

In [11]:
#fin.to_csv('/tf/notebooks/code_for_pub/fp_files/fps_topo.csv', header=True, index=True)

In [25]:
fin.drop(columns=['smiles'], inplace=True)

In [27]:
with open('/tf/notebooks/code_for_pub/fp_files/fps_topo_300bit.pickle','wb') as f:
    pickle.dump(fin, f)

In [20]:
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb.pickle','rb') as f:
    test = pickle.load(f)

In [23]:
sorted(test.map(len), reverse=False)

[8,
 8,
 8,
 8,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,